In [1]:
import pandas as pd

In [2]:
# this notebook calculates and adds relevant ratios to our data to use as features
feats_target = pd.read_csv('./data/features_target_before_ratios.csv')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (3,5,6,7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# ratios/features to use:
# EPS, PE, PEG, market cap, price to book, change in working capital
# R&D, SGA, debt to equity
# extras: change in revenue,

feats_target.shape

(19898, 68)

In [4]:
feats_target['EPS'] = feats_target['Net Income'] / feats_target['Shares (Diluted)']

In [5]:
feats_target['PE'] = feats_target['Adj. Close'] / feats_target['EPS']

In [6]:
feats_target['EPS Growth Rate'] = 0
for i in range(1, len(feats_target)):
    feats_target.loc[i, 'EPS Growth Rate'] = (feats_target.loc[i, 'EPS'] - feats_target.loc[i - 1, 'EPS']) / abs(
        feats_target.loc[i - 1, 'EPS'])

<ipython-input-6-4583a4affe1f>:3: RuntimeWarning: divide by zero encountered in double_scalars
  feats_target.loc[i, 'EPS Growth Rate'] = (feats_target.loc[i, 'EPS'] - feats_target.loc[i - 1, 'EPS']) / abs(


In [7]:
# delete growth rate between companies
changes = feats_target['Ticker'].shift() != feats_target['Ticker']
feats_target['EPS Growth Rate'][changes] = None

<ipython-input-7-9975e0c61d9f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feats_target['EPS Growth Rate'][changes] = None


In [8]:
feats_target['PEG'] = feats_target['PE'] / feats_target['EPS Growth Rate']

In [9]:
feats_target['Market Cap'] = feats_target['Shares (Diluted)'] * feats_target['Adj. Close']

In [10]:
feats_target['Price to Book'] = feats_target['Market Cap'] / feats_target['Total Equity']

In [11]:
feats_target['Debt to Equity'] = feats_target['Total Liabilities'] / feats_target['Total Equity']

In [12]:
# merge in industries for PE ratio

In [13]:
companies = pd.read_csv('./data/us-companies.csv', sep=';')

In [14]:
companies.drop(columns=['Company Name', 'SimFinId'], inplace=True)

In [15]:
feats_target_industry = pd.merge(feats_target, companies, how='outer',
                                 left_on='Ticker', right_on='Ticker')

feats_target_industry.columns

Index(['Unnamed: 0', 'Ticker', 'SimFinId', 'Currency', 'Fiscal Year',
       'Fiscal Period', 'Report Date', 'Publish Date', 'Restated Date',
       'Shares (Basic)', 'Shares (Diluted)', 'Net Income/Starting Line',
       'Depreciation & Amortization', 'Non-Cash Items',
       'Change in Working Capital', 'Change in Accounts Receivable',
       'Change in Inventories', 'Change in Accounts Payable',
       'Change in Other', 'Net Cash from Operating Activities',
       'Change in Fixed Assets & Intangibles',
       'Net Change in Long Term Investment',
       'Net Cash from Acquisitions & Divestitures',
       'Net Cash from Investing Activities', 'Dividends Paid',
       'Cash from (Repayment of) Debt', 'Cash from (Repurchase of) Equity',
       'Net Cash from Financing Activities', 'Net Change in Cash',
       'Abnormal Gains (Losses)', 'Cost of Revenue', 'Gross Profit',
       'Income (Loss) from Continuing Operations',
       'Income Tax (Expense) Benefit, Net', 'Interest Expense, N

In [16]:
cleaned = feats_target_industry[
    ['Ticker', 'SimFinId', 'IndustryId', 'Fiscal Year', 'EPS', 'PE', 'EPS Growth Rate', 'PEG', 'Market Cap',
     'Price to Book', 'Debt to Equity', 'Change in Working Capital', 'Selling, General & Administrative',
     'Research & Development', 'Adj. Close']]

In [17]:
print('Entries before removing all nans:', cleaned.shape[0])

# get rid of entries where all relevant features are null
cleaned = cleaned[cleaned['SimFinId'].notna() | cleaned['Fiscal Year'].notna() | cleaned['EPS'].notna() |
                  cleaned['PE'].notna() | cleaned['EPS Growth Rate'].notna() | cleaned['PEG'].notna() |
                  cleaned['Market Cap'].notna() | cleaned['Price to Book'].notna() | cleaned['Debt to Equity'].notna() |
                  cleaned['Change in Working Capital'].notna() | cleaned['Selling, General & Administrative'].notna() |
                  cleaned['Research & Development'].notna()]

print('Entries after removing all nans:', cleaned.shape[0])

Entries before removing all nans: 20045
Entries after removing all nans: 13067


In [21]:
sorted_by_close = cleaned.sort_values('Adj. Close', ascending=False)
# cleaned.to_csv('final.csv')